In [ ]:
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!pip install pytesseract==0.3.9
!pip install ftfy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 4 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (5,144 kB/s)
debconf: unable to initia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.3 MB/s 


In [ ]:
import json
import pytesseract
import cv2
import numpy as np
import sys
import re
import os
from PIL import Image, ImageOps
import ftfy
import io
from google.colab.patches import cv2_imshow

In [ ]:
def detect_angle_front(image):
    mask = np.zeros(image.shape, dtype=np.uint8)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    adaptive = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,15,4)

    cnts = cv2.findContours(adaptive, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for c in cnts:
        area = cv2.contourArea(c)
        if area < 45000 and area > 20:
            cv2.drawContours(mask, [c], -1, (255,255,255), -1)

    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    # cv2_imshow(mask)
    h, w = mask.shape
    
    # Horizontal
    if w > h:
        left = mask[0:h, 0:0+w//4]
        right = mask[0:h, 3*w//4:]
        left_pixels = cv2.countNonZero(left)
        right_pixels = cv2.countNonZero(right)
        return 0 if left_pixels <= right_pixels else 180
    # Vertical
    else:
        top = mask[0:0+h//4, 0:w]
        bottom = mask[3*h//4:, 0:w]
        top_pixel = cv2.countNonZero(top)
        bottom_pixel = cv2.countNonZero(bottom)
        return 90 if top_pixel <= bottom_pixel else 270

def detect_angle_back(image):
    mask = np.zeros(image.shape, dtype=np.uint8)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    adaptive = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,15,4)

    cnts = cv2.findContours(adaptive, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for c in cnts:
        area = cv2.contourArea(c)
        if area < 45000 and area > 20:
            cv2.drawContours(mask, [c], -1, (255,255,255), -1)

    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    # cv2_imshow(mask)
    h, w = mask.shape
    
    # Horizontal
    if w > h:
        left = mask[0:h, 0:0+w//2]
        right = mask[0:h, w//2:]
        left_pixels = cv2.countNonZero(left)
        right_pixels = cv2.countNonZero(right)
        # print(left_pixels)
        # print(right_pixels)
        return 0 if left_pixels <= right_pixels else 180
    # Vertical
    else:
        top = mask[0:0+h//2, 0:w]
        bottom = mask[h//2:, 0:w]
        top_pixels = cv2.countNonZero(top)
        bottom_pixels = cv2.countNonZero(bottom)
        return 90 if top_pixels <= bottom_pixels else 270

image_front = cv2.imread('aadhar270.jpg')
image_back = cv2.imread('back180.jpg')
angle_front = detect_angle_front(image_front)
angle_back = detect_angle_back(image_back)
print(angle_front)
print(angle_back)

270
180


In [ ]:
def adhaar_read_data(textFront,textBack):
  resFront=textFront.split()
  resBack=textBack.splitlines()
  
  resBack = [ i for i in resBack if i != '']
  
  text3 = resBack[3:7]

  name = None
  dob = None
  adh = None
  sex = None
  address=""
  temp=""
  
  text0 = []
  text1 = []
  
  lines = textFront.split('\n')
  for lin in lines:
      s = lin.strip()
      s = lin.replace('\n','')
      s = s.rstrip()
      s = s.lstrip()
      text1.append(s)

  if 'female' in textFront.lower():
      sex = "FEMALE"
  else:
      sex = "MALE"
  
  text1 = list(filter(None, text1))
  text0 = text1[:]
  
  try:

      # Cleaning first names
      name = text0[1]
      name = name.rstrip()
      name = name.lstrip()
      name = name.replace("8", "B")
      name = name.replace("0", "D")
      name = name.replace("6", "G")
      name = name.replace("1", "I")
      name = re.sub('[^a-zA-Z] +', ' ', name)

      # Cleaning DOB
      dob = text0[2][-10:]
      dob = dob.rstrip()
      dob = dob.lstrip()
      dob = dob.replace('l', '/')
      dob = dob.replace('L', '/')
      dob = dob.replace('I', '/')
      dob = dob.replace('i', '/')
      dob = dob.replace('|', '/')
      dob = dob.replace('\"', '/1')
      dob = dob.replace(":","")
      dob = dob.replace(" ", "")

      # Cleaning Adhaar number details
      aadhar_number=''
      for word in resFront:
          if len(word) == 4 and word.isdigit():
              aadhar_number=aadhar_number  + word + ' '
      if len(aadhar_number)>=14:
          print("Aadhar number is :"+ aadhar_number)
      else:
          print("Aadhar number not read")
      adh=aadhar_number

      # temp=temp.join(text3)
      address=' '.join(text3)

  except:
      pass

  data={}
  data['Name'] = name
  data['Date of Birth'] = dob
  data['Adhaar Number'] = adh
  data['Sex'] = sex
  data['ID Type'] = "Adhaar"
  data['Address'] = address
  return data
  
im = Image.open(r"back180.jpg")
im = ImageOps.exif_transpose(im)
if angle_back != 0:
  im = im.rotate(angle_back,Image.NEAREST, expand = 1)
width, height = im.size
 
left = width/2
top = 5
right = width
bottom = height
 
im1 = im.crop((left, top, right, bottom))

open_cv_image = np.array(im1) 
dst = cv2.fastNlMeansDenoisingColored(open_cv_image, None, 10, 10, 7, 15)
config = ('-l eng --psm 1')
extractedbacktext = pytesseract.image_to_string(dst,config=config)
extractedbacktext_output = open('output2.txt', 'w', encoding='utf-8')
extractedbacktext_output.write(extractedbacktext)
extractedbacktext_output.close()
file = open('output2.txt', 'r', encoding='utf-8')
extractedbacktext = file.read()
extractedbacktext = ftfy.fix_text(extractedbacktext)
extractedbacktext = ftfy.fix_encoding(extractedbacktext)

from google.colab.patches import cv2_imshow
src = cv2.imread("aadhar270.jpg")
if angle_front == 0:
  img = src
if angle_front == 270:
  img = cv2.rotate(src, cv2.ROTATE_90_CLOCKWISE)
if angle_front == 180:
  img = cv2.rotate(src, cv2.ROTATE_180)
if angle_front == 90:
  img = cv2.rotate(src, cv2.ROTATE_90_COUNTERCLOCKWISE)
# cv2_imshow(img)
dst = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 15)
config = ('-l eng --psm 3')
extractedfronttext = pytesseract.image_to_string(dst,config=config)
extractedfronttext_output = open('output1.txt', 'w', encoding='utf-8')
extractedfronttext_output.write(extractedfronttext)
extractedfronttext_output.close()
file = open('output1.txt', 'r', encoding='utf-8')
extractedfronttext = file.read()
extractedfronttext = ftfy.fix_text(extractedfronttext)
extractedfronttext = ftfy.fix_encoding(extractedfronttext)

data = adhaar_read_data(extractedfronttext,extractedbacktext)

try:
    to_unicode = unicode
except NameError:
    to_unicode = str
with io.open('info.json', 'w', encoding='utf-8') as outfile:
    data = json.dumps(data, indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(data))

with open('info.json', encoding = 'utf-8') as data:
    data_loaded = json.load(data)

print("\n---------- ADHAAR Details ----------")
print("\nADHAAR Number: ",data_loaded['Adhaar Number'])
print("\nName: ",data_loaded['Name'])
print("\nDate Of Birth: ",data_loaded['Date of Birth'])
print("\nSex: ",data_loaded['Sex'])
print("\nAddress:",data_loaded['Address'])
print("\n------------------------------------")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.


Aadhar number is :2767 3063 7751 

---------- ADHAAR Details ----------

ADHAAR Number:  2767 3063 7751 

Name:  Harshit Biser

Date Of Birth:  S/10/1999

Sex:  MALE

Address: Address: S/O Sanjay Singh, Balrampur House, 33 - Ujwal, Mahanagar'' Bareilly, Bareilly, :

------------------------------------
